# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 30 2022 12:00AM,238328,10,MET210654,"Methapharm, Inc.",Released
1,May 27 2022 4:31PM,238329,20,ALUR366934652,Alumier Labs Inc.,Released
2,May 27 2022 4:31PM,238329,20,ALUR613091136,Alumier Labs Inc.,Released
3,May 27 2022 4:31PM,238329,20,ALUR655900322,Alumier Labs Inc.,Released
4,May 27 2022 4:31PM,238329,20,ALUR786010677,Alumier Labs Inc.,Released
5,May 27 2022 4:31PM,238329,20,ALUR852395487,Alumier Labs Inc.,Released
6,May 27 2022 4:31PM,238329,20,ALUR067643150,Alumier Labs Inc.,Released
7,May 27 2022 4:31PM,238329,20,ALUR276134484,Alumier Labs Inc.,Released
8,May 27 2022 4:31PM,238329,20,ALUR495288314,Alumier Labs Inc.,Released
9,May 27 2022 4:31PM,238329,20,ALU0006956,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,238311,Released,1
20,238313,Released,1
21,238325,Released,1
22,238328,Released,1
23,238329,Released,25


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238311,NaN,2.0,1.0
238313,NaN,NaN,1.0
238325,NaN,NaN,1.0
238328,NaN,NaN,1.0
238329,NaN,NaN,25.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238212,1.0,0.0,1.0
238253,0.0,0.0,2.0
238255,0.0,0.0,1.0
238256,0.0,0.0,1.0
238264,21.0,9.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238212,1,0,1
238253,0,0,2
238255,0,0,1
238256,0,0,1
238264,21,9,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238212,1,0,1
1,238253,0,0,2
2,238255,0,0,1
3,238256,0,0,1
4,238264,21,9,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238212,1,,1
1,238253,,,2
2,238255,,,1
3,238256,,,1
4,238264,21,9,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 30 2022 12:00AM,238328,10,"Methapharm, Inc."
1,May 27 2022 4:31PM,238329,20,Alumier Labs Inc.
26,May 27 2022 3:47PM,238325,21,"NBTY Global, Inc."
27,May 27 2022 2:58PM,238311,10,ISDIN Corporation
30,May 27 2022 2:58PM,238313,102,"Senseonics, Incorporated"
31,May 27 2022 2:56PM,238310,10,"Nextsource Biotechnology, LLC"
34,May 27 2022 2:45PM,238306,10,ISDIN Corporation
52,May 27 2022 2:05PM,238300,19,ACG North America LLC
53,May 27 2022 1:53PM,238299,16,American International Chemical
54,May 27 2022 1:31PM,238297,19,Eli Lilly and Company


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 30 2022 12:00AM,238328,10,"Methapharm, Inc.",,,1
1,May 27 2022 4:31PM,238329,20,Alumier Labs Inc.,,,25
2,May 27 2022 3:47PM,238325,21,"NBTY Global, Inc.",,,1
3,May 27 2022 2:58PM,238311,10,ISDIN Corporation,,2,1
4,May 27 2022 2:58PM,238313,102,"Senseonics, Incorporated",,,1
5,May 27 2022 2:56PM,238310,10,"Nextsource Biotechnology, LLC",,,3
6,May 27 2022 2:45PM,238306,10,ISDIN Corporation,,6,12
7,May 27 2022 2:05PM,238300,19,ACG North America LLC,,,1
8,May 27 2022 1:53PM,238299,16,American International Chemical,,,1
9,May 27 2022 1:31PM,238297,19,Eli Lilly and Company,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 30 2022 12:00AM,238328,10,"Methapharm, Inc.",1,,
1,May 27 2022 4:31PM,238329,20,Alumier Labs Inc.,25,,
2,May 27 2022 3:47PM,238325,21,"NBTY Global, Inc.",1,,
3,May 27 2022 2:58PM,238311,10,ISDIN Corporation,1,2,
4,May 27 2022 2:58PM,238313,102,"Senseonics, Incorporated",1,,
5,May 27 2022 2:56PM,238310,10,"Nextsource Biotechnology, LLC",3,,
6,May 27 2022 2:45PM,238306,10,ISDIN Corporation,12,6,
7,May 27 2022 2:05PM,238300,19,ACG North America LLC,1,,
8,May 27 2022 1:53PM,238299,16,American International Chemical,1,,
9,May 27 2022 1:31PM,238297,19,Eli Lilly and Company,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 30 2022 12:00AM,238328,10,"Methapharm, Inc.",1,,
1,May 27 2022 4:31PM,238329,20,Alumier Labs Inc.,25,,
2,May 27 2022 3:47PM,238325,21,"NBTY Global, Inc.",1,,
3,May 27 2022 2:58PM,238311,10,ISDIN Corporation,1,2,
4,May 27 2022 2:58PM,238313,102,"Senseonics, Incorporated",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 30 2022 12:00AM,238328,10,"Methapharm, Inc.",1.0,NaN,NaN
1,May 27 2022 4:31PM,238329,20,Alumier Labs Inc.,25.0,NaN,NaN
2,May 27 2022 3:47PM,238325,21,"NBTY Global, Inc.",1.0,NaN,NaN
3,May 27 2022 2:58PM,238311,10,ISDIN Corporation,1.0,2.0,NaN
4,May 27 2022 2:58PM,238313,102,"Senseonics, Incorporated",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 30 2022 12:00AM,238328,10,"Methapharm, Inc.",1.0,0.0,0.0
1,May 27 2022 4:31PM,238329,20,Alumier Labs Inc.,25.0,0.0,0.0
2,May 27 2022 3:47PM,238325,21,"NBTY Global, Inc.",1.0,0.0,0.0
3,May 27 2022 2:58PM,238311,10,ISDIN Corporation,1.0,2.0,0.0
4,May 27 2022 2:58PM,238313,102,"Senseonics, Incorporated",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1906275,21.0,17.0,22.0
102,238313,1.0,0.0,0.0
16,476581,1.0,1.0,0.0
17,238256,1.0,0.0,0.0
19,476597,2.0,0.0,0.0
20,714860,38.0,21.0,0.0
21,238325,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1906275,21.0,17.0,22.0
1,102,238313,1.0,0.0,0.0
2,16,476581,1.0,1.0,0.0
3,17,238256,1.0,0.0,0.0
4,19,476597,2.0,0.0,0.0
5,20,714860,38.0,21.0,0.0
6,21,238325,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,21.0,17.0,22.0
1,102,1.0,0.0,0.0
2,16,1.0,1.0,0.0
3,17,1.0,0.0,0.0
4,19,2.0,0.0,0.0
5,20,38.0,21.0,0.0
6,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,21.0
1,102,Released,1.0
2,16,Released,1.0
3,17,Released,1.0
4,19,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,16,17,19,20,21
Status,,,,,,,
Completed,22.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,17.0,0.0,1.0,0.0,0.0,21.0,0.0
Released,21.0,1.0,1.0,1.0,2.0,38.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,16,17,19,20,21
0,Completed,22.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,17.0,0.0,1.0,0.0,0.0,21.0,0.0
2,Released,21.0,1.0,1.0,1.0,2.0,38.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,16,17,19,20,21
0,Completed,22.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,17.0,0.0,1.0,0.0,0.0,21.0,0.0
2,Released,21.0,1.0,1.0,1.0,2.0,38.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()